# Route Inspection Problem
### (Chinese Postman Problem)

Find the shortest closed path which visits every edge of a (connected) undirected graph.

### Context

A **Eulerian circuit** should exist in the graph: a closed traversal which visits every edge *exactly once*.

If such a circuit does not exist, generate that condition from the given graph: 

>Find the minimal set of the graph's edges to duplicate such that the resulting multigraph **does** have a Eulerian circuit.

The subset of edges must have the minimum possible total weight, or if edge weight is a non-factor, the smallest number of edges.

### Algorithm (generalised)

1. Find all nodes with an odd [degree](https://en.wikipedia.org/wiki/Degree_(graph_theory)).

2. Account for and minimise backtracking:

> add edges to the graph such that odd-degree nodes become even
    
> any edges added must be duplicates of graph's original edges
    
> the added set of edges must have minimal total weight (np-hard)
    
3. Find the Eulerian circuit in the regenerated multigraph.

### Implementation

Gather dependencies:

In [9]:
# stdlib dependencies
import itertools
import copy

# external libraries
import networkx as nx
import pandas   as pd
import matplotlib.pyplot as plt

Obtain external data (in a somewhat gorier way than normal):

In [13]:
import io
import requests

resp = requests.get('https://gist.githubusercontent.com/brooksandrew/e570c38bcc72a8d102422f2af836513b/raw/89c76b2563dbc0e88384719a35cba0dfc04cd522/edgelist_sleeping_giant.csv')
data = pd.read_csv(io.StringIO(resp.content.decode('utf-8')))

Our edgelist appears as such:

In [14]:
data

,node1,node2,trail,distance,color,estimate
0,rs_end_north,v_rs,rs,0.30,red,0
1,v_rs,b_rs,rs,0.21,red,0
2,b_rs,g_rs,rs,0.11,red,0
3,g_rs,w_rs,rs,0.18,red,0
4,w_rs,o_rs,rs,0.21,red,0
...,...,...,...,...,...,...
118,v_bv,b_bv,bv,0.08,purple,0
119,g_gy2,w_gy2,gy2,0.05,yellowgreen,0
120,w_gy2,b_gy2,gy2,0.03,yellowgreen,1
121,b_gy2,o_gy2,gy2,0.07,yellowgreen,0
